In [1]:
import csv
import os
import torch
import optree
import pyiqa
import numpy as np
# https://pypi.org/project/pyiqa/

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [3]:
# set input dir
input_dir = './OutputSurvey/'
content_dir = './ContentImages/'

# list all available metrics
#print(pyiqa.list_models())

In [4]:
#FR Metrics
#fid_metric = pyiqa.create_metric('fid',device=device)
fsim_metric = pyiqa.create_metric('fsim',device=device)
lpips_metric = pyiqa.create_metric('lpips',device=device)
psnr_metric = pyiqa.create_metric('psnr',device=device)
ssim_metric = pyiqa.create_metric('ssim',device=device)

#NR Metrics
brisque_metric = pyiqa.create_metric('brisque',device=device)
niqe_metric = pyiqa.create_metric('niqe',device=device)
piqe_metric = pyiqa.create_metric('piqe',device=device)

frmetric_block = [fsim_metric, lpips_metric, psnr_metric, ssim_metric]
nrmetric_block = [brisque_metric, niqe_metric, piqe_metric]

Loading pretrained model LPIPS from /home/andy/.cache/torch/hub/pyiqa/LPIPS_v0.1_alex-df73285e.pth


In [6]:
with open('image_quality_survey.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    #Create the header
    headerblock = ['file name', 'content source', 'art style', 'network']
    for item in frmetric_block:
        headerblock.append(item.metric_name)
    for item in nrmetric_block:
        headerblock.append(item.metric_name)
    csvwriter.writerow(headerblock)

    #Create each row
    files = os.listdir(input_dir)
    for file in files:
        if not file.endswith('.jpeg'):
            continue
        filepath = input_dir + file
        content = file.split('_')[2]
        art_style = file.split('_')[1]
        network = file.split('_')[0]
        contentpath = content_dir + content
        row = [file, content, art_style, network]
        for item in frmetric_block:
            row.append(item(filepath, contentpath).item())
        for item in nrmetric_block:
            row.append(item(filepath).item())
        csvwriter.writerow(row)
print("finished")

finished


In [19]:
print("Metric   Lower is Better?")
for metric in np.concatenate((frmetric_block,nrmetric_block)):
    print(f"{metric.metric_name:<8} {metric.lower_better}")



Metric   Lower is Better?
fsim     False
lpips    True
psnr     False
ssim     False
brisque  True
niqe     True
piqe     True


In [13]:
filepath

'./Output6/SqueezeNet_VG2_Autumn.jpeg'